In [1]:
#%pip install super-gradients
#%pip install torch
#%pip install supervision
#%pip install roboflow
#%pip install --upgrade

In [2]:
from roboflow import Roboflow
rf = Roboflow(api_key="NNsjGsIb23GWj3zGDggC")
project = rf.workspace("yolo-hz3ua").project("yolo-fj4s3")
dataset = project.version(2).download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics<=8.0.20 is required but found version=8.0.168, to fix: `pip install ultralytics<=8.0.20`


Extracting Dataset Version Zip to YOLO-2 in yolov8:: 100%|██████████| 612/612 [00:00<00:00, 1403.35it/s]


In [3]:
import torch
from super_gradients.training import Trainer

DEVICE = 'cuda' if torch.cuda.is_available() else "cpu"
MODEL_ARCH = 'yolo_nas_l'
ATCH_SIZE = 8
MAX_EPOCHS = 15
CHECKPOINT_DIR = './checkpoints'
EXPERIMENT_NAME = project.name.lower().replace(" ", "_")

from super_gradients.training import Trainer

trainer = Trainer(experiment_name=EXPERIMENT_NAME, ckpt_root_dir=CHECKPOINT_DIR)

LOCATION = dataset.location
CLASSES = sorted(project.classes.keys())

dataset_params = {
    'data_dir': LOCATION,
    'train_images_dir':'train/images',
    'train_labels_dir':'train/labels',
    'val_images_dir':'valid/images',
    'val_labels_dir':'valid/labels',
    'test_images_dir':'test/images',
    'test_labels_dir':'test/labels',
    'classes': CLASSES
}
print(dataset_params['classes'])


[2023-09-06 14:50:17] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it


The console stream is logged into C:\Users\mateu\sg_logs\console.log


[2023-09-06 14:50:18] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-09-06 14:50:19] WARNING - redirects.py - NOTE: Redirects are currently not supported in Windows or MacOs.
[2023-09-06 14:50:22] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-09-06 14:50:22] WARNING - export.py - Failed to import pytorch_quantization
[2023-09-06 14:50:22] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-09-06 14:50:22] WARNING - env_sanity_check.py - Failed to verify operating system: Deci officially supports only Linux kernels. Some features may not work as expected.


In [4]:
from super_gradients.training.dataloaders.dataloaders import (coco_detection_yolo_format_train, coco_detection_yolo_format_val)

train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 8,
        'num_workers': 1
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size': 8,
        'num_workers': 1
    }
)

[2023-09-06 14:50:23] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.


['tumor']


Indexing dataset annotations: 100%|██████████| 210/210 [00:01<00:00, 150.94it/s]
[2023-09-06 14:50:24] INFO - detection_dataset.py - Dataset Initialization in progress. `cache_annotations=True` causes the process to take longer due to full dataset indexing.
Indexing dataset annotations:  73%|███████▎  | 44/60 [00:00<00:00, 133.69it/s]

In [5]:
from super_gradients.training import models

model = models.get(
    MODEL_ARCH, 
    num_classes=len(dataset_params['classes']), 
    pretrained_weights="coco"
)

Indexing dataset annotations: 100%|██████████| 60/60 [00:00<00:00, 140.81it/s]
[2023-09-06 14:50:25] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.
[2023-09-06 14:50:26] INFO - checkpoint_utils.py - Successfully loaded pretrained weights for architecture yolo_nas_l


In [6]:
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

train_params = {
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-6,
    "lr_warmup_epochs": 3,
    "initial_lr": 5e-4,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.1,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.0001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    "max_epochs": MAX_EPOCHS,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}

In [7]:
print(DEVICE)
trainer.train(
    model=model, 
    training_params=train_params, 
    train_loader=train_data, 
    valid_loader=val_data
)

cuda
The console stream is now moved to ./checkpoints\yolo/console_set06_14_50_26.txt


[2023-09-06 14:50:30] INFO - sg_trainer.py - Using EMA with params {'decay': 0.9, 'decay_type': 'threshold'}
[2023-09-06 14:50:40] INFO - sg_trainer_utils.py - TRAINING PARAMETERS:
    - Mode:                         Single GPU
    - Number of GPUs:               1          (1 available on the machine)
    - Dataset size:                 210        (len(train_set))
    - Batch size per GPU:           8          (batch_size)
    - Batch Accumulate:             1          (batch_accumulate)
    - Total batch size:             8          (num_gpus * batch_size)
    - Effective Batch size:         8          (num_gpus * batch_size * batch_accumulate)
    - Iterations per epoch:         26         (len(train_loader))
    - Gradient updates per epoch:   26         (len(train_loader) / batch_accumulate)

[2023-09-06 14:50:40] INFO - sg_trainer.py - Started training for 15 epochs (0/14)

Train epoch 0:  27%|██▋       | 7/26 [03:09<08:34, 27.09s/it, PPYoloELoss/loss=3.93, PPYoloELoss/loss_cls=2

OutOfMemoryError: CUDA out of memory. Tried to allocate 50.00 MiB (GPU 0; 3.00 GiB total capacity; 5.21 GiB already allocated; 0 bytes free; 5.35 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
import torch

if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU disponível.")
    print("Dispositivo atual:", torch.cuda.get_device_name(0))  # 0 é o índice da GPU, pode ser diferente
    print("Memória VRAM disponível:", torch.cuda.get_device_properties(0).total_memory / (1024 ** 2), "MB")
else:
    print("GPU não disponível.")


GPU não disponível.
